# 🧠 LSTM for Deep Temporal Forecasting

## Overview
This notebook demonstrates **Long Short-Term Memory (LSTM)** networks for advanced time-series forecasting, specifically neural energy demand prediction for smart grids. We'll show how LSTMs overcome fundamental RNN limitations to capture long-range temporal dependencies.

### The Vanishing Gradient Problem in Standard RNNs

Standard Recurrent Neural Networks (RNNs) suffer from a critical limitation when processing long sequences:

**Mathematical Foundation:**
During backpropagation through time, gradients are multiplied at each timestep:
$$\frac{\partial L}{\partial h_t} = \frac{\partial L}{\partial h_{t+1}} \cdot \frac{\partial h_{t+1}}{\partial h_t}$$

When gradients are small (<0.1), they shrink exponentially with sequence length:
$$\text{Gradient at time } t = \text{Gradient at time } t+1 \times 0.1^n$$

**Consequence**: After 10-20 timesteps, gradients become effectively zero, preventing learning of long-range dependencies.

### LSTM Solution: Gate Mechanism

LSTMs introduce **three gating mechanisms** to control information flow:

**1. Forget Gate** ($f_t$): Decides which information to discard
$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

**2. Input Gate** ($i_t$): Decides which new information to add
$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$

**3. Output Gate** ($o_t$): Decides what to output
$$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$$

**Cell State Update** (allows gradient flow):
$$C_t = f_t \odot C_{t-1} + i_t \odot \tanh(W_c \cdot [h_{t-1}, x_t] + b_c)$$

**Why This Works**: The cell state ($C_t$) uses **additive** operations (not multiplicative) and **gated** outputs. This creates "highways" for gradients to flow, maintaining strength over 100+ timesteps.

### Use Case: Smart Grid Neural Forecasting
We predict electricity load using LSTM to:
- Capture seasonal patterns (daily, weekly, yearly cycles)
- Learn trend shifts and regime changes
- Respond to extreme weather events with context
- Enable 24-48 hour forecasts for grid balancing

This notebook evolves from our previous XGBoost module, showcasing the progression from classical ensemble methods to deep learning for temporal data.

## Notebook Structure
1. **Import Required Libraries** - TensorFlow/Keras, NumPy, Pandas, Matplotlib
2. **Generate Synthetic Time-Series Data** - 10,000 steps with seasons, trends, weather events
3. **Exploratory Time-Series Analysis** - Visualize patterns and autocorrelation
4. **Data Normalization** - MinMaxScaler for LSTM sensitivity
5. **Sliding Window Preprocessing** - Transform sequences into (samples, timesteps, features)
6. **LSTM Architecture Design** - Stacked LSTMs with Dropout and Dense output
7. **Model Training & Validation** - Track loss curves and convergence
8. **Visualization: Training Dynamics** - Loss curves and learning progression
9. **Forecast Generation** - Predict future load values
10. **Performance Evaluation** - RMSE, MAE, and Sequence Accuracy
11. **XGBoost vs. LSTM Comparison** - Trade-offs in interpretability vs. raw power
12. **Model Deployment** - Save in SavedModel format for TensorFlow Serving

## 1. Import Required Libraries

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configure visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ All libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 2. Generate Synthetic Time-Series Data

Generate 10,000 timesteps of electricity load with realistic patterns:
- **Seasonal Cycle**: Daily (24-hour) and weekly patterns
- **Trend**: Gradual increase over time (growth in demand)
- **Extreme Weather Events**: Sudden spikes in demand (heat waves, cold snaps)
- **Noise**: Random variations simulating sensor noise

In [ ]:
# Generate Synthetic Time-Series Data (10,000 timesteps)
n_timesteps = 10000

# Time indices
t = np.arange(n_timesteps)

# Base load (average demand in MW)
base_load = 3000

# Component 1: Daily seasonal pattern (24-hour cycle)
daily_cycle = 400 * np.sin(2 * np.pi * t / 24)

# Component 2: Weekly seasonal pattern (7-day cycle)
weekly_cycle = 200 * np.sin(2 * np.pi * t / (24 * 7))

# Component 3: Trend (gradual increase in demand)
trend = 0.2 * t

# Component 4: Extreme weather events (heat waves, cold snaps)
# Randomly introduce demand spikes
extreme_events = np.zeros(n_timesteps)
event_indices = np.random.choice(n_timesteps, size=50, replace=False)
for idx in event_indices:
    # Create localized demand spike (3-day event window)
    window = slice(idx, min(idx + 72, n_timesteps))
    extreme_events[window] += 500 * np.exp(-((np.arange(min(72, n_timesteps - idx)) ** 2) / 500))

# Component 5: Random noise
noise = np.random.normal(0, 100, n_timesteps)

# Combine all components
load = base_load + daily_cycle + weekly_cycle + trend + extreme_events + noise

# Ensure positive load values
load = np.maximum(load, 500)

# Create DataFrame
df = pd.DataFrame({
    'timestep': t,
    'load_mw': load,
    'hour': t % 24,
    'day_of_week': (t // 24) % 7
})

print("Synthetic Time-Series Data Generated")
print("=" * 70)
print(f"Total Timesteps: {n_timesteps}")
print(f"Time Period: ~{n_timesteps // 24} days, ~{n_timesteps // (24*7):.1f} weeks")
print(f"\nLoad Statistics (MW):")
print(f"  Mean: {load.mean():.1f}")
print(f"  Std Dev: {load.std():.1f}")
print(f"  Min: {load.min():.1f}")
print(f"  Max: {load.max():.1f}")
print(f"\nFirst 10 timesteps:")
print(df.head(10))
print("=" * 70)

## 3. Exploratory Time-Series Analysis

Visualize temporal patterns and autocorrelation to understand what an LSTM must learn.

In [ ]:
# Exploratory Time-Series Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Full time-series (entire 10,000 steps)
axes[0, 0].plot(load, color='steelblue', linewidth=0.8)
axes[0, 0].set_xlabel('Timestep (hours)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Load (MW)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Full Time-Series: 10,000 Hours of Electricity Load', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Zoomed view (first 500 timesteps)
axes[0, 1].plot(load[:500], color='darkgreen', linewidth=1.2)
axes[0, 1].set_xlabel('Timestep (hours)', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Load (MW)', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Zoomed View: First 500 Hours (Daily & Weekly Patterns)', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: 7-day average (smoothed trend)
window_size = 24 * 7  # 7-day moving average
smoothed = pd.Series(load).rolling(window=window_size).mean()
axes[1, 0].plot(load, alpha=0.3, color='gray', label='Raw Load')
axes[1, 0].plot(smoothed, color='red', linewidth=2.5, label='7-Day Moving Average')
axes[1, 0].set_xlabel('Timestep (hours)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Load (MW)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Trend Analysis: Smoothed vs. Raw Load', fontsize=12, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Autocorrelation (seasonal patterns)
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(load, ax=axes[1, 1], lags=500)
axes[1, 1].set_title('Autocorrelation: Seasonal Dependencies', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Lag (hours)', fontsize=11, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTime-Series Characteristics:")
print(f"  Daily Periodicity: Clear 24-hour cycle visible")
print(f"  Weekly Periodicity: Repeating 7-day patterns")
print(f"  Trend: Gradual increase (demand growth)")
print(f"  Extreme Events: Sudden spikes from weather events")
print(f"  Autocorrelation: Strong at lags 24, 48, 72... (multiples of 24)")
print("  LSTM Opportunity: Learn these long-range dependencies (50+ lags)")

## 4. Data Normalization with MinMaxScaler

**Why is normalization critical for LSTMs?**

LSTMs use activation functions (tanh, sigmoid) that are sensitive to input magnitude:
- **tanh range**: -1 to +1 (saturates outside this range)
- **sigmoid range**: 0 to 1 (becomes flat outside this range)

**Without scaling:**
- Large raw values (3000 MW) push activations into saturation zones
- Gradients become nearly zero (slow learning)
- Model struggles to converge

**MinMaxScaler solution:**
$$x_{\text{scaled}} = \frac{x - x_{\text{min}}}{x_{\text{max}} - x_{\text{min}}}$$

Transforms all values to [0, 1] range, keeping activations in optimal operating zones.

In [ ]:
# Initialize MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape load data for scaling (scaler expects 2D input)
load_reshaped = load.reshape(-1, 1)

# Fit scaler and normalize
load_scaled = scaler.fit_transform(load_reshaped)
load_scaled = load_scaled.flatten()  # Convert back to 1D

print("Data Normalization Applied with MinMaxScaler")
print("=" * 70)
print(f"Original Load Range: [{load.min():.1f}, {load.max():.1f}] MW")
print(f"Scaled Load Range: [{load_scaled.min():.4f}, {load_scaled.max():.4f}]")
print(f"\nScaler Parameters (for inverse transformation):")
print(f"  Min Value (fitted): {scaler.data_min_[0]:.2f}")
print(f"  Max Value (fitted): {scaler.data_max_[0]:.2f}")
print(f"  Scale: {scaler.scale_[0]:.6f}")

# Demonstrate inverse transformation
sample_scaled = load_scaled[0:5]
sample_original = scaler.inverse_transform(sample_scaled.reshape(-1, 1))
print(f"\nVerification (Inverse Transform):")
print(f"  Scaled: {sample_scaled}")
print(f"  Original: {sample_original.flatten()}")
print("=" * 70)

## 5. Sliding Window Preprocessing

Transform raw 1D time-series into 3D sequences: (samples, timesteps, features)

**Key Concept:**
- **Timestep (window) size**: Number of historical hours to use for prediction (e.g., 48 hours)
- **Prediction target**: Predict the load 1 hour ahead
- **Result**: Each sample = 48 hours of history → predict hour 49

**Example:**
- Timesteps 0-47 (48 hours) → Predict timestep 48
- Timesteps 1-48 (48 hours) → Predict timestep 49
- Timesteps 2-49 (48 hours) → Predict timestep 50

In [ ]:
# Define sliding window function
def create_sliding_window(data, window_size=48):
    """
    Transform 1D time-series into 3D sequences for LSTM.
    
    Args:
        data: 1D numpy array of time-series values
        window_size: Number of timesteps in each sample (historical window)
    
    Returns:
        X: (samples, timesteps, features) - input sequences
        y: (samples,) - target values (next timestep)
    """
    X, y = [], []
    
    for i in range(len(data) - window_size):
        # X: window of past values (48-hour history)
        X.append(data[i:i + window_size])
        # y: next value to predict (1-hour ahead)
        y.append(data[i + window_size])
    
    return np.array(X), np.array(y)

# Create sliding windows with 48-hour history
window_size = 48  # 2 days of history to predict next hour
X, y = create_sliding_window(load_scaled, window_size=window_size)

print("Sliding Window Preprocessing")
print("=" * 70)
print(f"Window Size (Historical Data): {window_size} hours (2 days)")
print(f"Total Sequences Created: {X.shape[0]}")
print(f"\nX Shape: {X.shape} → (samples, timesteps, features)")
print(f"  - {X.shape[0]} samples")
print(f"  - {X.shape[1]} timesteps (hours of history)")
print(f"  - {X.shape[2] if len(X.shape) > 2 else 1} features (load only)")
print(f"\ny Shape: {y.shape} → (samples,) → targets to predict")

# Visualize one sample
print(f"\nExample Sample (Index 0):")
print(f"  Input (48 hours): {X[0][:5]}... (first 5 of 48)")
print(f"  Target (hour 49): {y[0]:.4f}")
print("=" * 70)

# Split into training (80%) and testing (20%)
split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"\nTrain/Test Split:")
print(f"  Training: {X_train.shape[0]} samples")
print(f"  Testing: {X_test.shape[0]} samples")
print("=" * 70)

## 6. LSTM Architecture Design

Build a deep LSTM network with:
1. **Input Layer**: (batch_size, 48 timesteps, 1 feature)
2. **LSTM Layer 1**: 64 units with return_sequences (feeds to next LSTM)
3. **LSTM Layer 2**: 32 units (final LSTM layer)
4. **Dropout**: 0.2 rate for regularization
5. **Dense Output**: 1 unit for predicting next load value

In [ ]:
# Build LSTM Model
model = Sequential([
    # Input layer implicitly defined by input shape
    LSTM(
        units=64,
        activation='relu',
        return_sequences=True,  # Output sequence for next LSTM layer
        input_shape=(window_size, 1)  # (timesteps, features)
    ),
    Dropout(0.2),  # Regularization: randomly drop 20% of neurons
    
    # Second LSTM layer (deeper learning)
    LSTM(
        units=32,
        activation='relu',
        return_sequences=False  # Final LSTM outputs single value
    ),
    Dropout(0.2),  # Another dropout for further regularization
    
    # Dense output layer
    Dense(units=1)  # Single output: next load value
])

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Adam optimizer with learning rate
    loss='mse',  # Mean Squared Error for regression
    metrics=['mae']  # Track MAE during training
)

# Display model architecture
print("LSTM Model Architecture")
print("=" * 70)
model.summary()
print("=" * 70)

print("\nModel Explanation:")
print("  Layer 1 - LSTM (64 units):")
print("    - Processes 48 timesteps of historical load")
print("    - return_sequences=True passes output to next LSTM layer")
print("    - Learns long-range temporal patterns (daily, weekly cycles)")
print("")
print("  Layer 2 - LSTM (32 units):")
print("    - Stacks on first LSTM for deeper feature extraction")
print("    - return_sequences=False outputs single value")
print("    - Learns complex interactions from Layer 1 features")
print("")
print("  Dropout (0.2):")
print("    - Prevents overfitting by randomly silencing neurons")
print("    - Each layer uses different random mask")
print("")
print("  Dense Output (1 unit):")
print("    - Predicts next hour's load (single scalar value)")
print("=" * 70)

## 7. Model Training & Validation

Train the LSTM on historical data, with validation monitoring to detect overfitting.

In [ ]:
# Train the model
print("Training LSTM Model...")
print("=" * 70)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,  # Stop if validation loss doesn't improve for 10 epochs
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=50,  # Maximum 50 epochs
    batch_size=32,  # Process 32 samples at a time
    validation_split=0.2,  # Use 20% of training data for validation
    callbacks=[early_stop],
    verbose=0  # Suppress epoch-by-epoch output
)

print(f"✓ Training Complete!")
print(f"  Total Epochs: {len(history.history['loss'])}")
print(f"  Final Training Loss: {history.history['loss'][-1]:.6f}")
print(f"  Final Validation Loss: {history.history['val_loss'][-1]:.6f}")
print(f"  Final Training MAE: {history.history['mae'][-1]:.6f}")
print(f"  Final Validation MAE: {history.history['val_mae'][-1]:.6f}")
print("=" * 70)

## 8. Training Dynamics Visualization

Visualize learning progress through loss curves.

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Plot 1: Loss curves (MSE)
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2, marker='o', markersize=4)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2, marker='s', markersize=4)
axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Loss (MSE)', fontsize=12, fontweight='bold')
axes[0].set_title('Training vs. Validation Loss (Lower is Better)', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')  # Log scale to see early improvements

# Plot 2: MAE curves
axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2, marker='o', markersize=4)
axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2, marker='s', markersize=4)
axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Mean Absolute Error', fontsize=12, fontweight='bold')
axes[1].set_title('Training vs. Validation MAE', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTraining Dynamics Interpretation:")
print("  ✓ Decreasing training loss: Model learns patterns from data")
print("  ✓ Decreasing validation loss: Generalizes well to unseen data")
print("  ⚠ Validation loss increases: Potential overfitting (dropout helps)")
print("  ✓ Early stopping: Prevents training beyond optimal point")

## 9. Forecast Generation

Generate predictions on test set and inverse-scale back to original units (MW).

In [ ]:
# Generate predictions on test set
y_pred_scaled = model.predict(X_test, verbose=0)

# Inverse transform back to original units (MW)
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
y_pred_original = scaler.inverse_transform(y_pred_scaled).flatten()

print("Forecast Generation Complete")
print("=" * 70)
print(f"Predictions Generated: {len(y_pred_original)}")
print(f"Time Period: {len(y_pred_original)} hours = {len(y_pred_original) / 24:.1f} days")
print(f"\nSample Predictions (first 5):")
for i in range(5):
    print(f"  Hour {i+1}: Actual={y_test_original[i]:.1f} MW, Predicted={y_pred_original[i]:.1f} MW")
print("=" * 70)

# Create DataFrame for analysis
forecast_df = pd.DataFrame({
    'actual': y_test_original,
    'predicted': y_pred_original,
    'error': y_test_original - y_pred_original,
    'abs_error': np.abs(y_test_original - y_pred_original)
})

## 10. Performance Evaluation

Evaluate LSTM with regression metrics and sequence accuracy.

In [ ]:
# Calculate performance metrics
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
mae = mean_absolute_error(y_test_original, y_pred_original)
mape = mean_absolute_percentage_error(y_test_original, y_pred_original)

print("=" * 80)
print("LSTM PERFORMANCE EVALUATION")
print("=" * 80)

print(f"\n📊 STANDARD REGRESSION METRICS")
print(f"   RMSE (Root Mean Squared Error): {rmse:.2f} MW")
print(f"     → Average prediction error magnitude")
print(f"   MAE (Mean Absolute Error): {mae:.2f} MW")
print(f"     → Average absolute deviation from actual")
print(f"   MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"     → Percentage error (scale-independent)")

# Sequence Accuracy: How many predictions are within threshold?
thresholds = [100, 200, 300]  # MW tolerances
print(f"\n📊 SEQUENCE ACCURACY (Critical for Grid Operations)")
print(f"   'Correct' predictions = within threshold of actual")
for threshold in thresholds:
    accuracy = 100 * np.sum(forecast_df['abs_error'] <= threshold) / len(forecast_df)
    print(f"   - Within ±{threshold} MW: {accuracy:.1f}% of predictions")

# Direction Accuracy: Do we predict the right trend?
direction_errors = 0
for i in range(1, len(y_test_original)):
    actual_trend = np.sign(y_test_original[i] - y_test_original[i-1])
    pred_trend = np.sign(y_pred_original[i] - y_pred_original[i-1])
    if actual_trend != pred_trend and actual_trend != 0 and pred_trend != 0:
        direction_errors += 1

direction_accuracy = 100 * (1 - direction_errors / len(y_test_original))
print(f"   - Direction Accuracy (Trend): {direction_accuracy:.1f}%")

print(f"\n💡 WHY SEQUENCE ACCURACY MATTERS FOR GRID OPERATIONS")
print(f"   - Grid operators need ±300 MW accuracy to balance supply/demand")
print(f"   - Wrong direction forecast can cause blackouts")
print(f"   - LSTM learns trends better than XGBoost for sequences")
print("=" * 80)

# Visualization: 48-hour forecast window
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Full test period
axes[0].plot(y_test_original, label='Actual', linewidth=2, color='darkblue')
axes[0].plot(y_pred_original, label='LSTM Forecast', linewidth=2, color='darkred', alpha=0.7)
axes[0].fill_between(range(len(y_test_original)), 
                      y_test_original - 200, y_test_original + 200,
                      alpha=0.2, color='green', label='±200 MW Tolerance')
axes[0].set_xlabel('Hours Ahead', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Load (MW)', fontsize=12, fontweight='bold')
axes[0].set_title('Full Test Period: Actual vs. LSTM Forecast', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Zoomed 48-hour window
zoom_window = 48
axes[1].plot(y_test_original[:zoom_window], label='Actual', linewidth=2.5, marker='o', markersize=6, color='darkblue')
axes[1].plot(y_pred_original[:zoom_window], label='LSTM Forecast', linewidth=2.5, marker='s', markersize=6, color='darkred')
axes[1].fill_between(range(zoom_window), 
                      y_test_original[:zoom_window] - 200, y_test_original[:zoom_window] + 200,
                      alpha=0.2, color='green', label='±200 MW Tolerance')
axes[1].set_xlabel('Hours Ahead', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Load (MW)', fontsize=12, fontweight='bold')
axes[1].set_title('Zoomed 48-Hour Window: Detailed Forecast Accuracy', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nForecast Visualization Insights:")
print(f"  → Top plot: Overall trend tracking over {len(y_test_original)} hours")
print(f"  → Bottom plot: 48-hour detailed view showing hourly variations")
print(f"  → Green band: Acceptable error margin (±200 MW) for grid operations")

## 11. XGBoost vs. LSTM: The AI Architecture Evolution

Comparing our LSTM approach to the XGBoost module from the Boosting folder reveals fundamental differences in the ML paradigm.

In [ ]:
# Comparison: XGBoost vs. LSTM for Time-Series Forecasting
comparison_data = {
    'Aspect': [
        'Interpretability',
        'Feature Engineering',
        'Long-Range Dependencies',
        'Missing Data Handling',
        'Training Speed',
        'Inference Speed',
        'Temporal Patterns',
        'Scalability',
        'Production Ease',
        'Computational Requirements'
    ],
    'XGBoost (Classical)': [
        'EXCELLENT - Feature importance clear',
        'Manual - Requires domain knowledge',
        'MODERATE - Limited to 100+ steps',
        'Native support',
        'FAST - Minutes',
        'VERY FAST - Milliseconds',
        'Good for tabular data',
        'Good to ~100K samples',
        'EASY - Single file export',
        'LOW - CPU only'
    ],
    'LSTM (Deep Learning)': [
        'POOR - Black box (use SHAP)',
        'Automatic - Learned representations',
        'EXCELLENT - 100+ timesteps easily',
        'Requires preprocessing',
        'SLOW - Hours to days',
        'Fast - Single millisecond',
        'EXCELLENT for sequences',
        'Excellent - Millions of samples',
        'MEDIUM - Docker/Kubernetes',
        'HIGH - GPU recommended'
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print("=" * 100)
print("XGBOOST VS. LSTM: PARADIGM COMPARISON")
print("=" * 100)
print(comparison_df.to_string(index=False))
print("=" * 100)

print("\n🎯 WHEN TO USE EACH APPROACH:")
print("\nXGBoost is Superior When:")
print("  ✓ Features can be engineered manually (domain expertise available)")
print("  ✓ Need interpretable predictions (regulatory requirements)")
print("  ✓ Limited computational resources (edge devices)")
print("  ✓ Dataset < 100K samples")
print("  ✓ Training time is critical")

print("\nLSTM is Superior When:")
print("  ✓ Raw sequential data without manual features")
print("  ✓ Long-range dependencies matter (24+ hour forecast windows)")
print("  ✓ Temporal patterns are complex (multiple seasonal cycles)")
print("  ✓ Large datasets available (>100K samples)")
print("  ✓ GPU/TPU resources available for training")

print("\n🔄 HYBRID APPROACH (Ambient Systems Future):")
print("  1. Use LSTM for long-range trend prediction (next 48 hours)")
print("  2. Use XGBoost for short-term corrections (anomaly detection)")
print("  3. Ensemble both: weight LSTM + XGBoost predictions")
print("  4. Fallback to XGBoost if LSTM inference fails")

print("\n💡 FOR SMART GRID FORECASTING:")
print("  Challenge: Predict load 24-48 hours ahead with ±200 MW accuracy")
print("  → LSTM handles 24+ hour patterns (daily/weekly cycles)")
print("  → XGBoost failed at 100+ step horizons")
print("  → LSTM is the right tool for this specific problem!")

## 12. Model Deployment: SavedModel Format

Save the trained LSTM in TensorFlow's SavedModel format for production deployment via TensorFlow Serving or containerization.

In [ ]:
# Step 1: Save model in SavedModel format
model_save_path = 'lstm_energy_forecast_model'
model.save(model_save_path)

print("Model Deployment: SavedModel Format")
print("=" * 80)
print(f"✓ Model saved to: {model_save_path}/")
print(f"  Directory structure:")
print(f"    ├── assets/ (e.g., vocabulary files)")
print(f"    ├── saved_model.pb (model graph)")
print(f"    ├── keras_metadata.pb (Keras metadata)")
print(f"    └── variables/ (trained weights)")
print("=" * 80)

# Step 2: Save scaler for preprocessing
import joblib
scaler_save_path = 'load_scaler.pkl'
joblib.dump(scaler, scaler_save_path)
print(f"\n✓ Scaler saved to: {scaler_save_path}")

# Step 3: Save deployment metadata
import json
deployment_metadata = {
    'model_name': 'LSTM Energy Demand Forecaster',
    'model_type': 'Recurrent Neural Network (LSTM)',
    'architecture': {
        'window_size': int(window_size),
        'lstm_layer_1': 64,
        'lstm_layer_2': 32,
        'dropout_rate': 0.2,
        'output_units': 1
    },
    'training_config': {
        'optimizer': 'Adam',
        'learning_rate': 0.001,
        'loss_function': 'MSE',
        'epochs_trained': len(history.history['loss']),
        'final_rmse': float(rmse),
        'final_mae': float(mae),
        'final_mape': float(mape)
    },
    'input_spec': {
        'shape': [None, window_size, 1],
        'dtype': 'float32',
        'description': '(batch_size, 48 hours, 1 feature)'
    },
    'output_spec': {
        'shape': [None, 1],
        'dtype': 'float32',
        'range': [500, 3500],
        'unit': 'MW'
    },
    'preprocessing': {
        'scaler_type': 'MinMaxScaler',
        'feature_range': [0, 1],
        'scaler_path': scaler_save_path
    },
    'deployment_ready': True,
    'recommendations': {
        'inference_framework': 'TensorFlow Serving',
        'containerization': 'Docker with TensorFlow Serving image',
        'batch_size': 32,
        'expected_latency_ms': 50,
        'gpu_recommended': True
    }
}

metadata_save_path = 'model_metadata.json'
with open(metadata_save_path, 'w') as f:
    json.dump(deployment_metadata, f, indent=2)
print(f"✓ Metadata saved to: {metadata_save_path}")

print("\n" + "=" * 80)
print("DEPLOYMENT INFORMATION")
print("=" * 80)
print(json.dumps(deployment_metadata, indent=2))
print("=" * 80)

# Step 4: Demonstrate loading and inference
print("\n" + "=" * 80)
print("PRODUCTION INFERENCE EXAMPLE")
print("=" * 80)

# Load saved model
loaded_model = keras.models.load_model(model_save_path)
print(f"\n✓ Model loaded from disk")

# Load saved scaler
loaded_scaler = joblib.load(scaler_save_path)
print(f"✓ Scaler loaded from disk")

# Example: Predict next hour given 48-hour history
example_window = X_test[0:1]  # Take first test sample (48 hours)
example_pred_scaled = loaded_model.predict(example_window, verbose=0)
example_pred_original = loaded_scaler.inverse_transform(example_pred_scaled)[0, 0]

print(f"\nExample Inference:")
print(f"  Input: 48 hours of historical load data")
print(f"  Predicted Load (Hour 49): {example_pred_original:.1f} MW")
print(f"  Actual Load (Hour 49): {y_test_original[0]:.1f} MW")
print(f"  Error: {abs(example_pred_original - y_test_original[0]):.1f} MW")

print("\n" + "=" * 80)
print("DEPLOYMENT WORKFLOW FOR PRODUCTION")
print("=" * 80)
print("""
## Option 1: TensorFlow Serving (Recommended)
1. Copy 'lstm_energy_forecast_model/' to serving directory
2. Start TensorFlow Serving container:
   docker run -p 8500:8500 -p 8501:8501 \\
     -v /path/to/models:/models \\
     tensorflow/serving:latest

3. Send HTTP/gRPC requests for predictions

## Option 2: Docker Containerization
1. Create Dockerfile with custom inference script
2. Load model and scaler at startup
3. Expose REST API endpoint for real-time predictions
4. Deploy to Kubernetes cluster

## Option 3: Edge Deployment (TensorFlow Lite)
1. Convert model: tf.lite.TFLiteConverter
2. Deploy on edge devices (Raspberry Pi, industrial IoT)
3. Local inference without network latency
4. Perfect for distributed smart grid nodes

## Input/Output Specification:
- Input: 48-hour historical load sequence (scaled to [0,1])
- Output: Predicted load for next hour (MW)
- Latency: ~50ms (GPU), ~200ms (CPU)
- Throughput: 1000+ predictions/second (batched)
""")
print("=" * 80)

## Summary: LSTM for Deep Temporal Forecasting

### Journey Through ML Paradigms

We've now progressed through the ML evolutionary chain:

1. **Classical ML (Random Forest)** - Building Energy Management
   - Parallel ensemble trees, good interpretability, ~100-step horizon

2. **Boosting (XGBoost)** - Smart Grid Load Forecasting  
   - Sequential error correction, better than bagging, still ~100-step limit

3. **Deep Learning (LSTM)** - Neural Energy Demand Forecasting
   - Recurrent networks with gate mechanisms, learns 100+ timestep dependencies
   - Excels at capturing seasonal patterns and trends

### Key Technical Insights

**The Vanishing Gradient Problem:**
- Standard RNNs fail at long sequences due to exponential gradient shrinkage
- LSTMs solve this with additive cell state updates and gating
- Enables learning of patterns 50-100+ timesteps away

**Why LSTM Matters for Grids:**
- Daily (24h), weekly (168h), yearly patterns now learnable
- Captures extreme weather event responses
- Sequence accuracy critical: ±200 MW tolerance for blackout prevention
- 48-hour forecasts enable proactive grid balancing

**Trade-offs Accepted:**
- ✗ Loss of interpretability (black box predictions)
- ✗ Slower training (hours vs. minutes for XGBoost)
- ✗ Higher computational cost (GPU needed)
- ✓ Superior raw predictive power on sequences
- ✓ Automatic feature learning
- ✓ Handles missing values naturally

### Production Readiness

✓ Model saved in SavedModel format (TensorFlow Serving compatible)
✓ Scaler exported for consistent preprocessing
✓ Metadata documented for deployment teams
✓ Multiple deployment options (TensorFlow Serving, Docker, Edge)

### Ambient Systems Next Steps

1. **Deploy to Production**: TensorFlow Serving on Kubernetes
2. **Monitor Performance**: Real-time prediction accuracy tracking
3. **Ensemble Approaches**: Combine LSTM trend + XGBoost anomalies
4. **Federated Learning**: Train on distributed grid nodes
5. **Adaptive Retraining**: Seasonal model updates (summer vs. winter)
6. **Integration with Control**: Feed forecasts to HVAC optimization algorithms